# En Yakın Komşu ALgoritması

* Tahminler gözlem benzerliğine göre yapılır
* çevrenizdeki beş kişinin ortalamasısınızdır

# KNN Basamakları
* Komşu sayısını belirle (k)
* Bilinmeyen nokta ile diğer tüm noktalar arası uzaklıoğı hesapla
* uzaklıkları sırala ve belirlene k sayısına göre en yakın olan k gözlemi seç
* Sınıflandırma ise en sıksınıf, regresyon ise ortalam değeri tahmin değeri olarak ver